In [ ]:
import pandas as pd
import numpy as np
#import tailer as tl
from scipy import signal
import matplotlib.pyplot as plt
import serial 
import time


In [ ]:
# In[] Lectura del documento

ps= serial.Serial('/dev/cu.usbmodem14201',9600)
cont=0
while True:
    time.sleep(0.5)
    df = pd.read_csv('/Applications/SavedData/OpenBCI-RAW-data_08.txt',delim_whitespace=True, names=('Tiempo', '1', '2','3','4','5','6','7','8'))
    
    #TODOS LOS CANALES 
    #data1=df.tail(600) #7500 muestras equivalente a 3 seg  765
    #data =  data1.loc[: ,"1":"8"]  #Tomar canal 8  
    #data["1"]= data["1"].str.replace(",","")  #Retirar la coma en B
    #data["2"]= data["2"].str.replace(",","")  #Retirar la coma en C
    #data["3"]= data["3"].str.replace(",","")  #Retirar la coma en D
    #data["4"]= data["4"].str.replace(",","")  #Retirar la coma en E
    #data["5"]= data["5"].str.replace(",","")  #Retirar la coma en F
    #data["6"]= data["6"].str.replace(",","")  #Retirar la coma en G
    #data["7"]= data["7"].str.replace(",","")  #Retirar la coma en H
    #data["8"]= data["8"].str.replace(",","")  #Retirar la coma en I
    #
    #data.values[:,0] = pd.to_numeric(data.values[:,0],downcast='signed')
    #data.values[:,1] = pd.to_numeric(data.values[:,1],downcast='signed')
    #data.values[:,2] = pd.to_numeric(data.values[:,2],downcast='signed')
    #data.values[:,3] = pd.to_numeric(data.values[:,3],downcast='signed')
    #data.values[:,4] = pd.to_numeric(data.values[:,4],downcast='signed')
    #data.values[:,5] = pd.to_numeric(data.values[:,5],downcast='signed')
    #data.values[:,6] = pd.to_numeric(data.values[:,6],downcast='signed')
    #data.values[:,7] = pd.to_numeric(data.values[:,7],downcast='signed')
    
    #Solo un canal
    dt = df.loc[: , "7"]
    dt1=dt.tail(500) #2 seg
    #dt1=dt.iloc[100:140000]
    data = dt1.str.replace(",","")
    data = pd.to_numeric(data,downcast='signed')
    np.nan_to_num(data,[])
        # In[] 
    fs= 250
    t = np.arange(1,len(data)+1,1)
    
    tsample = 1/fs
    # In[] Filtros
    f_low = 50                        
    f_high = 1                      
    # Filtro pasa bajas de 50 hz
    b, a = signal.butter(2, 2*f_low/fs, btype='low')
    filt= signal.filtfilt(b, a, data)
    #filtered1=signal.lfilter(b,a,sig)
    
    #mean_data = np.array(data)
    #for i in range(0,7):
    #   data2[:,i]=signal.filtfilt(b,a,data.values[:,i]);
    #Filtro pasa altas de 1 hz
    b1, a1 = signal.butter(2, 2*f_high/fs, btype='high')
    filt1= signal.filtfilt(b1, a1, filt)
    #Filtro Notch
    fstart=(58)/fs*2;# Cutoff frequencies
    fstop=(62)/fs*2;# Cutoff frequencies
    b2, a2 = signal.butter(2,[fstart,fstop],'bandstop'); # Calculate filter coefficients
    filt2= signal.filtfilt(b2, a2, filt1)
    
    filt2=filt2/24
   
    # In[] Transformada de Fourier
    
    ft = np.abs(np.fft.fft(filt2)) #Magnitud 
    ft = ft[0:int(len(ft)/2)] 
    f = np.linspace(0,fs/2,len(ft))# Vector de frecuencias 
    
#    plt.figure()
#    ax1 = plt.subplot(2,1,1)
#    plt.plot(t,filt2)
#    plt.title('EEG'),plt.xlabel('Tiempo (s)'),plt.ylabel('Amplitud')
#    plt.grid()
#    ax1 = plt.subplot(2,1,2)
#    plt.plot(f,ft)
#    plt.title('Magnitud de la transformada de Fourier'),plt.xlabel('Frecuencia (Hz)'),plt.ylabel('Amplitud')
#    plt.grid()
#    plt.show()
    
    
    # In[] Detección onda alpha
    for i in range(len(f)): 
        f1= f[i,]
        if f1 >= 8 and f1<= 14:
            amp=ft[i,]
            print(amp)
            if amp >= 20500:
                ps.write(1)
                print("1")
            
            else: 
                ps.write(0)
#                print("0")
    # In[] Envio serial
    #    if bandera ==1:
    #        ps.write(1)
    #        cont=cont+1
    #        print(cont)
    #    elif  bandera ==0: 
    #        ps.write(0)


